# World University Rankings 2023 #

## STAT 301 Group Project 

### Introduction

Start with relevant background information on the topic to prepare those unfamiliar for the rest of your proposal.

Formulate one or two questions for investigation and detail the dataset that will be utilized to address these questions.

Additionally, align your question/objectives with the existing literature. To contextualize your study, include a minimum of two scientific publications (these should be listed in the References section).



### Methods and Results

In this section, you will include:

**a) “Exploratory Data Analysis (EDA)”**

- Demonstrate that the dataset can be read into R.
- Clean and wrangle your data into a tidy format.
- Plot the relevant raw data, tailoring your plot to address your question.
  - Make sure to explore the association of the explanatory variables with the response.
- Any summary tables that are relevant to your analysis.
- Be sure not to print output that takes up a lot of screen space.
- Your EDA must be comprehensive with high quality plots.

**b) “Methods: Plan”**

- Describe in written English the methods you used to perform your analysis from beginning to end, and narrate the code that does the analysis.
- If included, describe the “Feature Selection” process and how and why you choose the covariates of your final model.
- Make sure to interpret/explain the results you obtain. It’s not enough to just say, “I fitted a linear model with these covariates, and my R-square is 0.87”.
  - If inference is the aim of your project, a detailed interpretation of your fitted model is required, as well as a discussion of relevant quantities (e.g., are the coefficients significant? How does the model fit the data)?
  - A careful model assessment must be conducted.
  - If prediction is the project's aim, describe the test data used or how it was created.
- Ensure your tables and/or figures are labelled with a figure/table number.

In [1]:
library(tidyverse)
library(repr)
library(broom)
library(GGally)
library(car)
library(rsample)
library(leaps)
library(glmnet)

Warning message:
“package ‘dplyr’ was built under R version 4.3.2”
Warning message:
“package ‘stringr’ was built under R version 4.3.2”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
“package ‘GGally’ was built under R version 4.3.2”
Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2

Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


The following object is masked from ‘package:purrr’:

    som

In [2]:
# part a
#loading and fixing column names
university_data <- read_csv("uni_rankings_2023.csv")
colnames(university_data) <- c("university_rank", "name_of_university", "location", "no_of_student", "no_of_student_per_staff", 
                               "international_student", "female_male_ratio", "overall_score", "teaching_score", "research_score",
                               "citations_score","industry_income_score", "international_outlook_score")
head(university_data)

# changing data from chr to dbl
university_data_cleaned <- university_data |>
mutate(international_student = as.numeric(gsub("%", "", international_student)) / 100,
      female_male_ratio = as.numeric(sub(":.*", "", female_male_ratio))/as.numeric(sub(".*:", "", female_male_ratio)),
      overall_score = as.numeric(overall_score),
      teaching_score = as.numeric(teaching_score),
      research_score = as.numeric(research_score),
      citations_score = as.numeric(citations_score),
      industry_income_score = as.numeric(industry_income_score),
      international_outlook_score = as.numeric(international_outlook_score)) |>
drop_na()

head(university_data_cleaned)

Rows: 2341 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (11): University Rank, Name of University, Location, International Stude...
dbl  (1): No of student per staff
num  (1): No of student

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


university_rank,name_of_university,location,no_of_student,no_of_student_per_staff,international_student,female_male_ratio,overall_score,teaching_score,research_score,citations_score,industry_income_score,international_outlook_score
<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,University of Oxford,United Kingdom,20965,10.6,42%,48 : 52,96.4,92.3,99.7,99.0,74.9,96.2
2,Harvard University,United States,21887,9.6,25%,50 : 50,95.2,94.8,99.0,99.3,49.5,80.5
3,University of Cambridge,United Kingdom,20185,11.3,39%,47 : 53,94.8,90.9,99.5,97.0,54.2,95.8
3,Stanford University,United States,16164,7.1,24%,46 : 54,94.8,94.2,96.7,99.8,65.0,79.8
5,Massachusetts Institute of Technology,United States,11415,8.2,33%,40 : 60,94.2,90.7,93.6,99.8,90.9,89.3
6,California Institute of Technology,United States,2237,6.2,34%,37 : 63,94.1,90.9,97.0,97.3,89.8,83.6


Warning message:
“There were 6 warnings in `mutate()`.
The first warning was:
ℹ In argument: `overall_score = as.numeric(overall_score)`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 5 remaining warnings.”


university_rank,name_of_university,location,no_of_student,no_of_student_per_staff,international_student,female_male_ratio,overall_score,teaching_score,research_score,citations_score,industry_income_score,international_outlook_score
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,University of Oxford,United Kingdom,20965,10.6,0.42,0.9230769,96.4,92.3,99.7,99.0,74.9,96.2
2,Harvard University,United States,21887,9.6,0.25,1.0000000,95.2,94.8,99.0,99.3,49.5,80.5
3,University of Cambridge,United Kingdom,20185,11.3,0.39,0.8867925,94.8,90.9,99.5,97.0,54.2,95.8
3,Stanford University,United States,16164,7.1,0.24,0.8518519,94.8,94.2,96.7,99.8,65.0,79.8
5,Massachusetts Institute of Technology,United States,11415,8.2,0.33,0.6666667,94.2,90.7,93.6,99.8,90.9,89.3
6,California Institute of Technology,United States,2237,6.2,0.34,0.5873016,94.1,90.9,97.0,97.3,89.8,83.6


### Part b

We will first fit the full linear model.

In [3]:
# Main Developer: Kaichi

# fitting the linear model
university_lm <- lm(overall_score ~ . - university_rank - name_of_university - location, data = university_data_cleaned)
uni_tidy_lm <- tidy(university_lm)
uni_tidy_lm

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1.850202e-02,3.456206e-02,0.53532757,5.933669e-01
no_of_student,-8.625404e-08,2.270879e-07,-0.37982662,7.047136e-01
no_of_student_per_staff,-3.538851e-04,3.015935e-04,-1.17338432,2.428551e-01
international_student,1.294070e-03,3.714650e-02,0.03483693,9.722649e-01
female_male_ratio,-1.823315e-02,9.996851e-03,-1.82388907,7.053866e-02
teaching_score,2.988413e-01,5.299324e-04,563.92339403,3.234071e-216
research_score,3.008783e-01,4.628687e-04,650.02951100,5.452256e-224
citations_score,3.005338e-01,3.213449e-04,935.23775731,6.825761e-244
industry_income_score,2.487489e-02,1.831504e-04,135.81670636,1.727166e-138


**Table 1**: Estimated Coefficients of `university_lm`

Now we will verify their vifs to identify if multicollinearity exists

In [4]:
# Main Developer: Kaichi

# check their vifs
university_vif <- vif(university_lm)
university_vif

no_of_student     no_of_student_per_staff 
                   1.421581                    1.459595 
      international_student           female_male_ratio 
                   3.005962                    1.594561 
             teaching_score              research_score 
                   9.428611                    9.267245 
            citations_score       industry_income_score 
                   1.616941                    1.537604 
international_outlook_score 
                   3.125979

Research score and teaching score have concerningly high variance inflation factors, indicating we should remove these variables from the model. However, upon closer inspection, the two variables are highly correlated to each other, thus, removing the variable with the highest vif should suffice.

In [5]:
# Main Developer: Kaichi

teaching_removed_lm <- lm(overall_score ~ . - university_rank - name_of_university - location - teaching_score, 
                          data = university_data_cleaned)
teaching_removed_vif <- vif(teaching_removed_lm)
teaching_removed_vif

no_of_student     no_of_student_per_staff 
                   1.383967                    1.388949 
      international_student           female_male_ratio 
                   2.999455                    1.446593 
             research_score             citations_score 
                   1.500528                    1.580512 
      industry_income_score international_outlook_score 
                   1.498008                    2.765207

However, using only the vif may not result in the best fitted model; thus, we will use regularization techniques to select the best variables to include in our final model. Since this data has a large number of covariates where multicollinearity could potentially be an issue and we want to find the best variables to predict future values of `overall_score`, we will run Lasso regularization to select our varaiables for the final model. 

First we must standaradize the data.

In [6]:
uni_standardized <- university_data_cleaned |>
mutate(no_of_student = (no_of_student - mean(no_of_student)) / sd(no_of_student),
       no_of_student_per_staff = (no_of_student_per_staff - mean(no_of_student_per_staff)) / sd(no_of_student_per_staff),
       international_student = (international_student - mean(international_student)) / sd(international_student),
       female_male_ratio = (female_male_ratio - mean(female_male_ratio)) / sd(female_male_ratio),
       teaching_score = (teaching_score - mean(teaching_score)) / sd(teaching_score),
       research_score = (research_score - mean(research_score)) / sd(research_score),
       citations_score = (citations_score - mean(citations_score)) / sd(citations_score),
       industry_income_score = (industry_income_score - mean(industry_income_score)) / sd(industry_income_score),
       international_outlook_score = (international_outlook_score - mean(international_outlook_score)) / sd(international_outlook_score)
       )
head(uni_standardized)

university_rank,name_of_university,location,no_of_student,no_of_student_per_staff,international_student,female_male_ratio,overall_score,teaching_score,research_score,citations_score,industry_income_score,international_outlook_score
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,University of Oxford,United Kingdom,-0.3983546,-0.5576823,1.30517312,-0.5662087,96.4,2.285459,2.138132,1.1657643,0.6875956,1.3080749
2,Harvard University,United States,-0.3360481,-0.6462550,0.01285609,-0.3501393,95.2,2.438543,2.100368,1.1926635,-0.6435130,0.3976797
3,University of Cambridge,United Kingdom,-0.4510651,-0.4956814,1.07711717,-0.6681283,94.8,2.199731,2.127342,0.9864362,-0.3972055,1.2848801
3,Stanford University,United States,-0.7227943,-0.8676867,-0.06316255,-0.7662730,94.8,2.401803,1.976287,1.2374955,0.1687777,0.3570888
5,Massachusetts Institute of Technology,United States,-1.0437199,-0.7702567,0.62100528,-1.2864402,94.2,2.187485,1.809048,1.2374955,1.5260891,0.9079649
6,California Institute of Technology,United States,-1.6639464,-0.9474021,0.69702393,-1.5093690,94.1,2.199731,1.992472,1.0133354,1.4684427,0.5774392


In [7]:
colnames(university_data_cleaned)

[1] "university_rank"             "name_of_university"         
 [3] "location"                    "no_of_student"              
 [5] "no_of_student_per_staff"     "international_student"      
 [7] "female_male_ratio"           "overall_score"              
 [9] "teaching_score"              "research_score"             
[11] "citations_score"             "industry_income_score"      
[13] "international_outlook_score"

Next we will split our data into training(70%) and testing(30%) data.

In [16]:
# Main Developer: Kaichi

data_split <- initial_split(uni_standardized, prop = 0.7) #splits the dataset into a 7:3 ratio
uni_training <- training(data_split) #creates training data which includes 70% of original data
uni_testing <- testing(data_split) #creates testing data which includes 30% of original data
head(uni_training)

university_rank,name_of_university,location,no_of_student,no_of_student_per_staff,international_student,female_male_ratio,overall_score,teaching_score,research_score,citations_score,industry_income_score,international_outlook_score
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
183,Autonomous University of Barcelona,Spain,0.3428357,-0.37167965,-0.6713117,1.0543121,55.6,-1.2171102,-1.14731042,0.4574184,-0.9841510,-0.3793456
66,Utrecht University,Netherlands,0.3833146,0.01804017,-0.8993677,1.4239046,68.2,-0.6537600,0.32007910,0.4663848,0.5723027,0.2991018
1,University of Oxford,United Kingdom,-0.3983546,-0.55768229,1.3051731,-0.5662087,96.4,2.2854585,2.13813155,1.1657643,0.6875956,1.3080749
9,Yale University,United States,-0.9040372,-0.97397391,-0.2912185,-0.1160640,91.4,2.3038286,1.76049454,0.9864362,-0.3552808,-0.1589951
69,University of North Carolina at Chapel Hill,United States,0.2851921,-0.79682854,-1.2794609,0.8830376,67.8,0.2402523,-0.01979421,0.6546793,-0.8269335,-1.8522143
37,London School of Economics and Political Science,United Kingdom,-1.0636553,-0.44253780,3.6617512,0.2740614,76.5,0.2586224,0.76784869,0.8160745,-1.2566614,1.1109193


**Table 2**: First 5 Rows of Training Data

In [9]:
head(uni_testing)

university_rank,name_of_university,location,no_of_student,no_of_student_per_staff,international_student,female_male_ratio,overall_score,teaching_score,research_score,citations_score,industry_income_score,international_outlook_score
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,University of Oxford,United Kingdom,20965,10.6,0.42,0.9230769,96.4,92.3,99.7,99.0,74.9,96.2
2,Harvard University,United States,21887,9.6,0.25,1.0000000,95.2,94.8,99.0,99.3,49.5,80.5
3,Stanford University,United States,16164,7.1,0.24,0.8518519,94.8,94.2,96.7,99.8,65.0,79.8
5,Massachusetts Institute of Technology,United States,11415,8.2,0.33,0.6666667,94.2,90.7,93.6,99.8,90.9,89.3
7,Princeton University,United States,8279,8.0,0.23,0.8518519,92.4,87.6,95.9,99.1,66.0,80.3
10,Imperial College London,United Kingdom,18545,11.2,0.61,0.6666667,90.4,82.8,90.8,98.3,59.8,97.5


**Table 3**: First 5 Rows of Testing Data

Now we will run Lasso regularization on `uni_training` since the training data is used to train the model to fit the data and penalize large coefficients. By running lasso on the training data we will be able to find the value of the L1 penalty term ($\lambda$) that provides the lowest cross-validation MSE.

In [10]:
# Main Developer: Kaichi

#run the lasso regression on the training set
uni_lasso<-
    cv.glmnet(uni_training %>% select(-university_rank, -name_of_university, -location, -overall_score) %>% as.matrix(), 
              uni_training$overall_score, 
              alpha=1)

uni_lasso


Call:  cv.glmnet(x = uni_training %>% select(-university_rank, -name_of_university,      -location, -overall_score) %>% as.matrix(), y = uni_training$overall_score,      alpha = 1) 

Measure: Mean-Squared Error 

    Lambda Index Measure      SE Nonzero
min 0.1498    46  0.1209 0.01802       5
1se 0.1498    46  0.1209 0.01802       5

Now that we ran the model, we will extract the coefficients of the model with the lowest cross-validation MSE and the names of the covariates.

In [11]:
# Main Developer: Kaichi

lasso_coef <-
    coef(uni_lasso, s = uni_lasso$lambda.min)

lasso_selected_covariates <-
    as_tibble(
        as.matrix(lasso_coef),
        rownames='covariate') %>%
        filter(covariate != '(Intercept)' & abs(s1) !=0) %>% 
        pull(covariate)
lasso_coef

10 x 1 sparse Matrix of class "dgCMatrix"
                                    s1
(Intercept)                 2.96995906
no_of_student               .         
no_of_student_per_staff     .         
international_student       .         
female_male_ratio           .         
teaching_score              0.27576747
research_score              0.31578478
citations_score             0.28621785
industry_income_score       0.01392209
international_outlook_score 0.06473458

**Table 4**: Coefficients of Model with Lowest MSE

In [12]:
# Main Developer: Kaichi

#obtain the selected covariates
lasso_selected_covariates

[1] "teaching_score"              "research_score"             
[3] "citations_score"             "industry_income_score"      
[5] "international_outlook_score"

Lasso regression should remove variables with high multicollinearity and is able to fit models on datasets with high multicollinearity. To verify this, we will check the variance inflation factor (VIF) of the lasso model

In [13]:
# Main Developer: Kaichi

lasso_vif <- vif(lm(overall_score ~ . , data = uni_training %>% 
        select(contains(lasso_selected_covariates), overall_score)))
lasso_vif

teaching_score              research_score 
                   7.401326                    7.614958 
            citations_score       industry_income_score 
                   1.192079                    1.236826 
international_outlook_score 
                   1.318297

Most of the variables selected in this model have low vif values indicating there is no issue of multicollinearity. However, `teaching_score` and `reasearch_score` both have high vif values which point to multicollinearity. Despite the high multicollinearity, the model could still be effective since lasso regression includeds variables that have high multicollinearity if they contribute significantly to the model. To verify the effectiveness of the model, we will fit an oridinary least squares model with the lasso selected variables and evaluate the $R^2$ value to determine the models effectiveness. To fit the OLS we will use the testing data which was kept separate from the training process as it provides an unbiased evaluation when comparing different models and a measure of how well the model does on unseen data. 

In [14]:
# Main Developer: Kaichi

#fit OLS model using lasso selected covariates
lasso_cov_model <- 
    lm(overall_score ~ .,
        data = uni_testing %>% 
                   select(contains(lasso_selected_covariates), overall_score))
summary(lasso_cov_model)


Call:
lm(formula = overall_score ~ ., data = uni_testing %>% select(contains(lasso_selected_covariates), 
    overall_score))

Residuals:
      Min        1Q    Median        3Q       Max 
-0.056492 -0.018862  0.001413  0.021904  0.057303 

Coefficients:
                             Estimate Std. Error t value Pr(>|t|)    
(Intercept)                 0.0117204  0.0692639   0.169    0.867    
teaching_score              0.2999584  0.0008189 366.297   <2e-16 ***
research_score              0.3000477  0.0006876 436.380   <2e-16 ***
citations_score             0.2997240  0.0008639 346.959   <2e-16 ***
industry_income_score       0.0250524  0.0003547  70.632   <2e-16 ***
international_outlook_score 0.0751303  0.0004260 176.371   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.03427 on 35 degrees of freedom
Multiple R-squared:      1,	Adjusted R-squared:      1 
F-statistic: 1.246e+06 on 5 and 35 DF,  p-value: < 2.2e-16


**Table 5**: Summary of OLS Model Using Lasso Selected Variables

We will now compare it to the full OLS model.

In [15]:
summary(university_lm)


Call:
lm(formula = overall_score ~ . - university_rank - name_of_university - 
    location, data = university_data_cleaned)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.084303 -0.025437  0.000312  0.022858  0.080683 

Coefficients:
                              Estimate Std. Error t value Pr(>|t|)    
(Intercept)                  1.850e-02  3.456e-02   0.535   0.5934    
no_of_student               -8.625e-08  2.271e-07  -0.380   0.7047    
no_of_student_per_staff     -3.539e-04  3.016e-04  -1.173   0.2429    
international_student        1.294e-03  3.715e-02   0.035   0.9723    
female_male_ratio           -1.823e-02  9.997e-03  -1.824   0.0705 .  
teaching_score               2.988e-01  5.299e-04 563.923   <2e-16 ***
research_score               3.009e-01  4.629e-04 650.030   <2e-16 ***
citations_score              3.005e-01  3.213e-04 935.238   <2e-16 ***
industry_income_score        2.487e-02  1.832e-04 135.817   <2e-16 ***
international_outlook_score  7.482e

**Table 6**: Summary of Full OLS Model

The adjusted $R^2$ in the full model is the same as the model with the lasso selected variables which implies that both the full model and model with lasso selected variables explain the variance `overall_score` equally than the model with both of the $R^2$ being 1. The $R^2$ being 1 implies that both models perfectly explain the variance of `overall_score`. Since the $R^2$ between the full model and lasso model was relatively the same, it justifies the variables selected with the lasso model since it implies that the variables removed did not make a significant impact to the models ability to explain the variance in `overall_score`. This result is also consistent with the result of the $p$-values in the full model as the variables that were removed in the lasso regression are also shown as insignificant in the full model.

### Discussion

In this section, you’ll interpret the results you obtained in the previous section with respect to the main question/goal of your project.

Summarize what you found and the implications/impact of your findings.
If relevant, discuss whether your results were what you expected to find.
Discuss how your model could be improved;
Discuss future questions/research this study could lead to.

### References

At least two citations of literature relevant to the project. The citation format is your choice – just be consistent. Make sure to cite the source of your data as well.